In [ ]:
#!pip install transformers
!pip install transformers tensorflow

In [ ]:
!pip install stanza

In [ ]:
import stanza
stanza.download('ar')  # Download the Arabic model
# Load Stanza pipeline for Arabic
nlp = stanza.Pipeline('ar', processors='tokenize,pos,ner')

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('kaggle/input/augoriginal'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# load the tweets dataset
df_tweets = pd.read_csv('/kaggle/input/augoriginal/augOriginal_1.csv')
# load the test dataset
df_tweets_test = pd.read_csv('/kaggle/input/val-test/testDS_1.csv')
# load the validation dataset
df_tweets_val = pd.read_csv('/kaggle/input/val-test/task2_dev.csv') 

In [ ]:
df_tweets['technique'].value_counts()

In [ ]:

df_tweets.drop(columns=["Unnamed: 0", "start", "end","text", "start_new","end_new"], inplace=True)

#df_tweets_test['augmented_text'] = df_tweets_test['text']
#df_tweets_test.drop(columns=["start", "end","Unnamed: 0", "text", "start_new","end_new"], inplace=True)

df_tweets_val['augmented_text'] = df_tweets_val['text']
df_tweets_val['augmented_full_text'] = df_tweets_val['full_text']
df_tweets_val.drop(columns=["start", "end","Unnamed: 0", "text", "start_new","end_new"], inplace=True)


df_all_tweets_list = [df_tweets,df_tweets_val]
df_all_tweets = pd.concat(df_all_tweets_list)


# First group tags Id wise
df_tags = df_all_tweets[['technique','augmented_full_text', 'augmented_text']]
classes = df_tags['technique'].unique()

'''
df_tags = df_tags.drop_duplicates(subset=['technique','augmented_full_text'], ignore_index = True)
df_tags = df_tags.groupby('augmented_full_text').apply(lambda x:x['technique'].values).reset_index(name='techniques')
'''

# Group by 'augmented_full_text' and aggregate 'technique' and 'augmented_text' as lists
grouped_df = df_tags.groupby('augmented_full_text').agg({
    'technique': list,
    'augmented_text': list
}).reset_index()

'''
# Define a regular expression pattern to match non-Arabic characters
non_arabic_pattern = r'[^\u0600-\u06FF\s]'

# Remove non-Arabic characters using the regular expression pattern
df_tags['augmented_full_text'] = df_tags['augmented_full_text'].str.replace(non_arabic_pattern, '', regex=True)

df_tags.shape
'''
df_new = grouped_df


In [ ]:
classes

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Fit and transform labels using MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes = classes)

labels = df_new['technique'].tolist()

labels_encoded = mlb.fit_transform(labels)

In [ ]:
x = df_new['augmented_full_text'].values
y = labels_encoded

In [ ]:

# First group tags Id wise
df_tweets_test['augmented_full_text'] = df_tweets_test['full_text']
#df_tweets_test.drop(columns=["start", "end","Unnamed: 0", "text", "start_new","end_new"], inplace=True)
test_tags = df_tweets_test[['technique','augmented_full_text']]
test_classes = test_tags['technique'].unique()
test_tags = test_tags.drop_duplicates(subset=['technique','augmented_full_text'], ignore_index = True)
test_tags = test_tags.groupby('augmented_full_text').apply(lambda x:x['technique'].values).reset_index(name='technique')


# Define a regular expression pattern to match non-Arabic characters
non_arabic_pattern = r'[^\u0600-\u06FF\s]'

# Remove non-Arabic characters using the regular expression pattern
test_tags['augmented_full_text'] = test_tags['augmented_full_text'].str.replace(non_arabic_pattern, '', regex=True)


test_labels = test_tags['technique'].tolist()

test_labels_encoded = mlb.fit_transform(test_labels)

#df_tweets_test['augmented_text'] = df_tweets_test['text']
#df_tweets_test.drop(columns=["start", "end","Unnamed: 0", "full_text" , "text", "start_new","end_new"], inplace=True)


In [ ]:

x_ts = test_tags['augmented_full_text'].values
y_ts = test_labels_encoded

In [ ]:
from sklearn.model_selection import train_test_split

# Define your random seed
RANDOM_SEED = 42

# First, split into training + validation and test sets
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=RANDOM_SEED)



In [ ]:
#!pip install transformers
from transformers import TFBertModel
# Load the BERT model
#bert_model = TFBertModel.from_pretrained('aubmindlab/bert-base-arabertv02')
#bert_model = TFBertModel.from_pretrained('aubmindlab/bert-base-arabertv02')
# Define the hyperparameters
max_seq_length = 100
num_classes = len(classes)
LEARNING_RATE = 2e-5   #0.001 => BERT  
EPOCHS = 100
BATCH_SIZE = 64

STEPS_PER_EPOCH = len(x_train)/BATCH_SIZE

In [ ]:
from transformers import BertTokenizer
# Tokenize and encode the training and testing data
tokenizer = BertTokenizer.from_pretrained('aubmindlab/bert-base-arabertv02')
# Tokenize and encode the training and testing data

# Tokenize and encode the training data
train_encodings = tokenizer(
    x_train.tolist(),  # Convert x_train to a list of strings
    truncation=True,
    padding='max_length',
    max_length=max_seq_length,
    return_tensors='tf'  # Return TensorFlow tensors
)

# Tokenize and encode the training data
val_encodings = tokenizer(
    x_val.tolist(),  # Convert x_train to a list of strings
    truncation=True,
    padding='max_length',
    max_length=max_seq_length,
    return_tensors='tf'  # Return TensorFlow tensors
)


# Tokenize and encode the training data
test_encodings = tokenizer(
    x_ts.tolist(),  # Convert x_train to a list of strings
    truncation=True,
    padding='max_length',
    max_length=max_seq_length,
    return_tensors='tf'  # Return TensorFlow tensors
)

# Define inputs for BERT tokenizer output
train_inputs = [train_encodings['input_ids'], train_encodings['attention_mask'], train_encodings['token_type_ids']]
val_inputs = [val_encodings['input_ids'], val_encodings['attention_mask'], val_encodings['token_type_ids']]
test_inputs = [test_encodings['input_ids'], test_encodings['attention_mask'], test_encodings['token_type_ids']]



In [ ]:
def get_pos_ner_features_stanza(texts):
    pos_features = []
    ner_features = []
    for text in texts:
        doc = nlp(text)
        pos_tags = [word.upos for sent in doc.sentences for word in sent.words]  # POS tags
        entities = [ent.type for ent in doc.ents]  # NER tags
        pos_features.append(pos_tags)
        ner_features.append(entities)
    return pos_features, ner_features

# Get POS and NER features for your training, validation, and test sets
pos_train, ner_train = get_pos_ner_features_stanza(x_train)
pos_val, ner_val = get_pos_ner_features_stanza(x_val)
pos_test, ner_test = get_pos_ner_features_stanza(x_ts)

In [ ]:
# Tokenize POS + NER features (Ensure tokenizer is defined as earlier)
pos_train_sequences = tokenizer.batch_encode_plus([" ".join(tags) for tags in pos_train], max_length=max_seq_length, truncation=True, padding='max_length', return_tensors='tf')
ner_train_sequences = tokenizer.batch_encode_plus([" ".join(tags) for tags in ner_train], max_length=max_seq_length, truncation=True, padding='max_length', return_tensors='tf')
pos_val_sequences = tokenizer.batch_encode_plus([" ".join(tags) for tags in pos_val], max_length=max_seq_length, truncation=True, padding='max_length', return_tensors='tf')
ner_val_sequences = tokenizer.batch_encode_plus([" ".join(tags) for tags in ner_val], max_length=max_seq_length, truncation=True, padding='max_length', return_tensors='tf')
pos_test_sequences = tokenizer.batch_encode_plus([" ".join(tags) for tags in pos_test], max_length=max_seq_length, truncation=True, padding='max_length', return_tensors='tf')
ner_test_sequences = tokenizer.batch_encode_plus([" ".join(tags) for tags in ner_test], max_length=max_seq_length, truncation=True, padding='max_length', return_tensors='tf')

# POS and NER inputs (tokenized sequences)
pos_train_input = pos_train_sequences['input_ids']
ner_train_input = ner_train_sequences['input_ids']
pos_val_input = pos_val_sequences['input_ids']
ner_val_input = ner_val_sequences['input_ids']
pos_test_input = pos_test_sequences['input_ids']
ner_test_input = ner_test_sequences['input_ids']

In [ ]:
train_inputs = [train_encodings['input_ids'], train_encodings['attention_mask'], train_encodings['token_type_ids']]
test_inputs = [test_encodings['input_ids'], test_encodings['attention_mask'], test_encodings['token_type_ids']]
val_inputs = [val_encodings['input_ids'], val_encodings['attention_mask'], val_encodings['token_type_ids']]



In [ ]:
import pathlib
import shutil
import tempfile

logdir = '/kaggle/working/logdir/tensorboard_logs_new'
shutil.rmtree(logdir, ignore_errors=True)

checkpoint_path = "/kaggle/working/checkpoint/training_2/cp-{epoch:04d}.ckpt"

In [ ]:

def get_callbacks(name):
  return [
    tfdocs.modeling.EpochDots(),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
    tf.keras.callbacks.TensorBoard(logdir+'/'+name),
    tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=30)
  ]

In [ ]:
import tensorflow as tf
from transformers import TFBertModel

# Define maximum sequence length
max_seq_length = 100  # Adjust according to your model/data

# Define the number of classes for your classification task
num_classes = 18  # Adjust based on your dataset

def create_model():
    # Load BERT model from Hugging Face
    bert_model = TFBertModel.from_pretrained('aubmindlab/bert-base-arabertv02')

    # Input layers for BERT
    input_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="attention_mask")
    token_type_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="token_type_ids")
    
    # BERT Model Outputs
    bert_outputs = bert_model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    pooled_output = bert_outputs.pooler_output  # [CLS] token output for classification tasks

    # POS and NER Inputs
    pos_input = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="pos_input")
    ner_input = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name="ner_input")

    # Embedding layers for POS and NER inputs
    pos_embedding = tf.keras.layers.Embedding(input_dim=50, output_dim=16, input_length=max_seq_length)(pos_input)
    ner_embedding = tf.keras.layers.Embedding(input_dim=50, output_dim=16, input_length=max_seq_length)(ner_input)

    # Flatten the embeddings
    pos_flat = tf.keras.layers.Flatten()(pos_embedding)
    ner_flat = tf.keras.layers.Flatten()(ner_embedding)

    # Concatenate BERT output with POS and NER embeddings
    combined_features = tf.keras.layers.Concatenate(axis=-1)([pooled_output, pos_flat, ner_flat])

    # Fully connected hidden layers
    hidden_layer1 = tf.keras.layers.Dense(128, activation='relu')(combined_features)
    hidden_layer2 = tf.keras.layers.Dense(64, activation='relu')(hidden_layer1)
    hidden_layer3 = tf.keras.layers.Dense(32, activation='relu')(hidden_layer2)

    # Output layer with sigmoid activation for multi-label classification
    output = tf.keras.layers.Dense(num_classes, activation='sigmoid')(hidden_layer3)

    # Define the model
    model = tf.keras.Model(
        inputs=[input_ids, attention_mask, token_type_ids, pos_input, ner_input],
        outputs=output
    )

    return model

# Create the model
model = create_model()

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Display the model summary to verify the architecture
model.summary()


In [ ]:
optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
# Define the weighted loss function
#loss = tf.keras.losses.BinaryCrossentropy(from_logits=False, pos_weight=18)
loss= 'binary_crossentropy'
#loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
accuracy=tf.keras.metrics.CategoricalAccuracy()

In [ ]:
def compile_and_fit(model_name, optimizer, loss_fn, accuracy, name, epochs_num, batch_size, steps_per_epoch):




    model_name.compile(optimizer=optimizer,
                loss=loss_fn,
                metrics=[
                  tf.keras.metrics.BinaryCrossentropy(
                      from_logits=False, name='binary_crossentropy'),
                  'accuracy'])

    model_name.summary()

    history = model.fit({
        'input_ids': train_inputs[0],
        'attention_mask': train_inputs[1],
        'token_type_ids': train_inputs[2],
        'pos_input': pos_train_input,
        'ner_input': ner_train_input
    },y_train,
                  validation_data=({
            'input_ids': val_inputs[0],
            'attention_mask': val_inputs[1],
            'token_type_ids': val_inputs[2],
            'pos_input': pos_val_input,
            'ner_input': ner_val_input
        },y_val),
                        epochs=epochs_num,batch_size=batch_size,steps_per_epoch = steps_per_epoch,
                        callbacks=get_callbacks(name),
                        verbose=1)

    return history

In [ ]:
size_histories = {}

In [ ]:
!pip install git+https://github.com/tensorflow/docs
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

In [ ]:
size_histories['ORIGINAL'] = compile_and_fit(model, optimizer, loss, accuracy, 'ORIGINAL', EPOCHS, BATCH_SIZE, STEPS_PER_EPOCH)

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(metric = 'binary_crossentropy', smoothing_std=10)
plotter.plot(size_histories)

#plt.ylim([-1, 2])

In [ ]:
# Ensure you're passing the correct input format for predictions (similar to training)
test_scores = model.predict({
    'input_ids': test_inputs[0],
    'attention_mask': test_inputs[1],
    'token_type_ids': test_inputs[2],
    'pos_input': pos_test_input,
    'ner_input': ner_test_input
})

# Convert scores to binary labels (0 or 1) based on a threshold of 0.5
test_pred_labels = (test_scores >= 0.5).astype(int)

# True labels
test_true_labels = y_ts.astype(int)

# Optionally, evaluate accuracy or other metrics
from sklearn.metrics import accuracy_score, f1_score, hamming_loss

# Calculate accuracy, F1 score, and Hamming loss
accuracy = accuracy_score(test_true_labels, test_pred_labels)
f1 = f1_score(test_true_labels, test_pred_labels, average='micro')
hamming = hamming_loss(test_true_labels, test_pred_labels)

print(f"Accuracy: {accuracy}")
print(f"F1 Score (micro): {f1}")
print(f"Hamming Loss: {hamming}")


In [ ]:
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, recall_score,classification_report

# Calculate F1 score
f1_macro = f1_score(test_true_labels, test_pred_labels, average='macro')
f1_micro = f1_score(test_true_labels, test_pred_labels, average='micro')

# Calculate confusion matrix, accuracy, and recall
confusion_mat = confusion_matrix(test_true_labels.flatten(), test_pred_labels.flatten())
accuracy = accuracy_score(test_true_labels.flatten(), test_pred_labels.flatten())
recall = recall_score(test_true_labels.flatten(), test_pred_labels.flatten(), average='macro')
classification_rep = classification_report(test_true_labels.flatten(), test_pred_labels.flatten())

from sklearn import metrics

print("Classification Report:")
print(classification_rep)
print("F1 macro score:", f1_macro)
print("F1 micro score:", f1_micro)
print("Confusion Matrix:")
print(confusion_mat)
print("Accuracy:", accuracy)
print("Recall:", recall)

In [ ]:
import pandas as pd

# Function to map binary labels to actual label names
def binary_to_text_labels(binary_labels, class_names):
    return [class_names[i] for i in range(len(binary_labels)) if binary_labels[i] == 1]

# Create a list to store the results
results = []

# Iterate over the test samples and collect the text, actual labels, and predicted labels
for i in range(len(x_ts)):  # test_texts is your list of test samples
    actual_label_names = binary_to_text_labels(test_true_labels[i], classes)  # using 'classes' array
    predicted_label_names = binary_to_text_labels(test_pred_labels[i], classes)  # using 'classes' array

    # Append the results to the list
    results.append({
        "Text": x_ts[i],
        "Actual Labels": ', '.join(actual_label_names),
        "Predicted Labels": ', '.join(predicted_label_names)
    })

# Convert the results list to a pandas DataFrame
results_df = pd.DataFrame(results)

# Display the DataFrame
results_df  # Use .head() to display the first few rows

# If you want to display the entire table, you can use:
# pd.set_option('display.max_rows', None)  # Uncomment this line to display all rows if needed
# display(results_df)

In [ ]:
# Save the model to a file
model.save("/kaggle/working/predict_resuls/BERT_1/augOriginal/cnn_lstm_attention_model.h5")


In [ ]:
# Save the results to a CSV file
results_df.to_csv("/kaggle/working/predict_resuls/BERT_1/augOriginal/results_df.csv", index=False)

# Alternatively, save to an Excel file
results_df.to_excel("/kaggle/working/predict_resuls/BERT_1/augOriginal/results_df.xlsx", index=False)

In [ ]:
# Save evaluation metrics to a text file
with open("/kaggle/working/predict_resuls/BERT_1/augOriginal/evaluation_metrics.txt", "w") as file:
    file.write("Classification Report:\n")
    file.write(classification_rep)
    file.write(f"\nF1 macro score: {f1_macro}\n")
    file.write(f"F1 micro score: {f1_micro}\n")
    file.write(f"\nConfusion Matrix:\n{confusion_mat}\n")
    file.write(f"Accuracy: {accuracy}\n")
    file.write(f"Recall: {recall}\n")

In [ ]:
import matplotlib.pyplot as plt

def plot_loss(size_histories):
    plt.figure(figsize=(10, 6))

    # Assuming 'ORIGINAL' is the key you want to plot
    key = 'ORIGINAL'

    history = size_histories[key]
    plt.plot(history.history['val_loss'], label=f'{key} Validation Loss')
    plt.plot(history.history['loss'], label=f'{key} Training Loss')

    plt.title("Training and Validation Loss over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.show()

plot_loss(size_histories)

In [ ]:
def plot_and_save_size_histories(size_histories, filename):
    plt.figure(figsize=(10, 6))

    for key, history in size_histories.items():
        plt.plot(history.history['val_loss'], label=f'{key} Validation Loss')
        plt.plot(history.history['loss'], '--', label=f'{key} Training Loss')

    plt.title("Training and Validation Loss over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)

    # Save the plot to a file
    plt.savefig(filename)
    plt.close()  # Close the plot to prevent displaying it if running in a loop

# Save the plot
plot_and_save_size_histories(size_histories, filename="/kaggle/working/predict_resuls/BERT_1/augOriginal/Training and Validation Loss over Epochs.png")

In [ ]:
# Plot training and validation accuracy
def plot_accuracy(size_histories):
    plt.figure(figsize=(10, 6))

    for key, history in size_histories.items():
        plt.plot(history.history['val_accuracy'], label=f'{key} Validation Accuracy')
        plt.plot(history.history['accuracy'], '--', label=f'{key} Training Accuracy')

    plt.title("Training and Validation Accuracy over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)
    plt.show()

plot_accuracy(size_histories)


In [ ]:
def plot_and_save_size_histories(size_histories, filename):
    plt.figure(figsize=(10, 6))

    for key, history in size_histories.items():
        plt.plot(history.history['val_accuracy'], label=f'{key} Validation Accuracy')
        plt.plot(history.history['accuracy'], '--', label=f'{key} Training Accuracy')

    plt.title("Training and Validation Accuracy over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)


    # Save the plot to a file
    plt.savefig(filename)
    plt.close()  # Close the plot to prevent displaying it if running in a loop

# Save the plot
plot_and_save_size_histories(size_histories, filename="/kaggle/working/predict_resuls/BERT_1/augOriginal/Training and Validation Accuracy over Epochs.png")

In [ ]:
# Plot training and validation binary crossentropy
def plot_binary_crossentropy(size_histories):
    plt.figure(figsize=(10, 6))

    for key, history in size_histories.items():
        plt.plot(history.history['val_binary_crossentropy'], label=f'{key} Validation Binary Crossentropy')
        plt.plot(history.history['binary_crossentropy'], '--', label=f'{key} Training Binary Crossentropy')

    plt.title("Training and Validation Binary Crossentropy over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Binary Crossentropy")
    plt.legend()
    plt.grid(True)
    plt.show()

plot_binary_crossentropy(size_histories)

In [ ]:
def plot_and_save_size_histories(size_histories, filename):
    plt.figure(figsize=(10, 6))

    for key, history in size_histories.items():
        plt.plot(history.history['val_binary_crossentropy'], label=f'{key} Validation Binary Crossentropy')
        plt.plot(history.history['binary_crossentropy'], '--', label=f'{key} Training Binary Crossentropy')

    plt.title("Training and Validation Binary Crossentropy over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Binary Crossentropy")
    plt.legend()
    plt.grid(True)

    # Save the plot to a file
    plt.savefig(filename)
    plt.close()  # Close the plot to prevent displaying it if running in a loop

# Save the plot
plot_and_save_size_histories(size_histories, filename="/kaggle/working/predict_resuls/BERT_1/augOriginal/Training and Validation Binary Crossentropy over Epochs.png")